<a href="https://colab.research.google.com/github/ravikumar-intelsat/AI-Langchain/blob/main/ChakraUIV3Migrations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
!pip install -qU langchain-google-vertexai
!pip install google-cloud-aiplatform langchain
!pip install faiss-cpu
!pip install  langchain-community

In [13]:
!pip install -qU langchain-google-genai


In [30]:
%pip install -qU langchain-text-splitters



In [7]:
!pip install -qU langchain-groq

In [ ]:
!pip install langchain-community

In [35]:
import os
import tempfile
from git import Repo
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.cloud import aiplatform
from langchain_google_vertexai import VertexAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
import vertexai
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAI as Gemini # Import Gemini from langchain_google_genai
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough




def clone_repo_to_memory(repo_url):
    """Clone a Git repository to a temporary directory."""
    temp_dir = tempfile.mkdtemp()
    Repo.clone_from(repo_url, temp_dir)
    return temp_dir


def get_files_recursively(folder_path, file_extensions=None):
    """Retrieve all files from a folder recursively, filtering by extensions."""
    file_list = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if not file_extensions or file.endswith(tuple(file_extensions)):
                file_list.append(os.path.join(root, file))
    return file_list


def split_file_content(file_path, chunk_size=500, chunk_overlap=50):
    """Split file content using RecursiveCharacterTextSplitter."""
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        ts_splitter = RecursiveCharacterTextSplitter.from_language( language=Language.TS, chunk_size=60, chunk_overlap=0)
    return ts_splitter.create_documents([content])


def process_repo(repo_url, file_extensions=None):
    """Clone a repo, split file contents recursively."""
    repo_path = clone_repo_to_memory(repo_url)
    files = get_files_recursively(repo_path, file_extensions)
    all_chunks = []
    for file in files:
        chunks = split_file_content(file)
        all_chunks.extend(chunks)
    return all_chunks


# Initialize Google Vertex AI
PROJECT_ID = "composed-maxim-443209-r4"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location=LOCATION)
# Set up Groq API key
embedding = VertexAIEmbeddings(model_name="text-embedding-004")
# model = ChatGroq(model="llama3-8b-8192")
os.environ["GROQ_API_KEY"] = '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'
model = ChatGroq(model="llama3-8b-8192")
repository_url = "https://github.com/chakra-ui/chakra-ui"
file_extensions = [".ipynb", ".js", ".ts", ".tsx", ".md"]
documents = process_repo(repository_url, file_extensions)
print(f"Total documents: {len(documents)}")
#write code to print all documents
# for document in documents:
#     print(document)

# write code load above documents to FAISS
vector_db = FAISS.from_documents(documents, embedding)

retriever = vector_db.as_retriever()

template = """Answer the question based only on the following context:
Now you are expert in React.js and chakra UI.
need to help in components from v2 to v3 using below context
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("""<Accordion>
  <AccordionItem>
    <h2>
      <AccordionButton>
        <Box as='span' flex='1' textAlign='left'>
          Section 1 title
        </Box>
        <AccordionIcon />
      </AccordionButton>
    </h2>
    <AccordionPanel pb={4}>
      Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod
      tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim
      veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea
      commodo consequat.
    </AccordionPanel>
  </AccordionItem>

  <AccordionItem>
    <h2>
      <AccordionButton>
        <Box as='span' flex='1' textAlign='left'>
          Section 2 title
        </Box>
        <AccordionIcon />
      </AccordionButton>
    </h2>
    <AccordionPanel pb={4}>
      Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod
      tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim
      veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea
      commodo consequat.
    </AccordionPanel>
  </AccordionItem>
</Accordion>""")
#

# # Store in Vector Database (FAISS)
# vector_db = FAISS.from_documents(documents, embedding)

# # Step 4: Create a QA chain
# qa_chain = RetrievalQA.from_chain_type(
#     llm=model,
#     retriever=vector_db.as_retriever()
# )

# # Step 5: Query the model
# query = "What is the main topic of the document?"
# results = qa_chain.run(query)

# # Example Chakra UI v2 component
# v2_component = """
# <Accordion>
#   <AccordionItem>
#     <h2>
#       <AccordionButton>
#         <Box as='span' flex='1' textAlign='left'>
#           Section 1 title
#         </Box>
#         <AccordionIcon />
#       </AccordionButton>
#     </h2>
#     <AccordionPanel pb={4}>
#       Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod
#       tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim
#       veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea
#       commodo consequat.
#     </AccordionPanel>
#   </AccordionItem>
#   <AccordionItem>
#     <h2>
#       <AccordionButton>
#         <Box as='span' flex='1' textAlign='left'>
#           Section 2 title
#         </Box>
#         <AccordionIcon />
#       </AccordionButton>
#     </h2>
#     <AccordionPanel pb={4}>
#       Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod
#       tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim
#       veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea
#       commodo consequat.
#     </AccordionPanel>
#   </AccordionItem>
# </Accordion>
# """

Total documents: 64375


"To upgrade this component from Chakra UI v2 to v3, I would suggest the following changes:\n\n1. Replace `Accordion` with `AccordionGroup`:\n```jsx\n<AccordionGroup>\n  // ...\n</AccordionGroup>\n```\nIn v3, `Accordion` has been renamed to `AccordionGroup`.\n\n2. Replace `AccordionItem` with `AccordionItem` (no change needed):\n```jsx\n<AccordionItem>\n  // ...\n</AccordionItem>\n```\nThe `AccordionItem` component remains the same in v3.\n\n3. Update `AccordionButton` and `AccordionPanel` to use the new `AccordionItem` API:\n```jsx\n<AccordionItem>\n  <AccordionItemHeading>\n    <h2>\n      <AccordionItemButton>\n        <Box as='span' flex='1' textAlign='left'>\n          Section 1 title\n        </Box>\n        <AccordionIcon />\n      </AccordionItemButton>\n    </h2>\n  </AccordionItemHeading>\n  <AccordionItemContent pb={4}>\n    Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod\n    tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim\n    ve